In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import serial
import time
import keyboard
from IPython.display import clear_output

In [2]:
# connect to Arduino

ser = serial.Serial('COM3', 9600, timeout=1) #check whether the Arduino is really connected with COM3 or adjust the code to the correct COM
                                             #check at settings and check whether Arduino is a connected device
time.sleep(2)

In [3]:
# read write the serial connection at a given sampling rate

def write_read(x):                  #this function takes an input x, which is a switch command and it returns data from the Arduino
    ser.write(bytes(x, 'utf-8'))    #pass the command to Arduino by the serial connection
    data = ser.readline()           #take data from Arduino
    return data                     #return the data to be processed further

In [4]:
values = []
timestamp = []

for j in range(0,5):
    num = '1'                           #power supply = True

    for i in range(0,5):
        clear_output(wait=True)
        value = write_read(num)
        dummy = value.decode().rstrip().split()                                     # transform the byte values that came in...
        values.append(dummy)
        timestamp.append(time.time())
        #dum0 = np.array([[int(j)/1024*5 for j in dummy[1:]]])
        print(pd.to_datetime(time.time(), unit= 's'),dummy, 
        #dum0[0],
              '            ', end = "\r")
    
    num = '0'

    for i in range(0,5):
        clear_output(wait=True)
        value = write_read(num)
        dummy = value.decode().rstrip().split()                                     # transform the byte values that came in...
        values.append(dummy)
        timestamp.append(time.time())
        # dum0 = np.array([[int(j)/1024*5 for j in dummy[1:]]])
        print(pd.to_datetime(time.time(), unit= 's'),value, 
        #dum0[0],
              '            ', end = "\r")
    
    
ser.close()       #close the serial port when the reading is stopped. If not done, reconnection with the Arduino is not possible    

In [5]:
df = pd.DataFrame(values, 
                  #index = pd.to_datetime(timestamp, unit = 's'), 
                  index = [x-timestamp[0] for x in timestamp],
                  columns = ['S','V0','V1','V2'])
df.iloc[:,1:] = df.iloc[:,1:].astype(int).div(1024/5)

In [6]:
#df.to_excel('kanweg.xlsx')

In [7]:
df

,S,V0,V1,V2
0.000000,1,4.985352,4.87793,0.0
1.023901,1,4.995117,4.960938,3.125
2.044923,1,4.995117,4.980469,4.267578
3.068438,1,4.995117,4.995117,4.711914
4.094829,1,4.995117,4.995117,4.87793
5.096222,0,0.004883,0.112305,4.941406
6.123712,0,0.0,0.03418,1.884766
7.151506,0,0.0,0.009766,0.717773
8.166280,0,0.0,0.0,0.273438
9.182987,0,0.0,0.0,0.107422
